Let us first do some dimension reduction of the word vector.

In [1]:
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm

In [2]:
gap_train= pd.read_pickle('./temp_result/train_kaggle_processed')
gap_test= pd.read_pickle('./temp_result/test_kaggle_processed')
gap_valid= pd.read_pickle('./temp_result/valid_kaggle_processed')
answer = pd.read_csv("https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-development.tsv",sep='\t')

这里是我的错,当时没考虑到这里没有标签不方便

In [3]:
gap_test["A-coref"] = answer["A-coref"]
gap_test["B-coref"] = answer["B-coref"]

In [4]:
gap_test.head()

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,B,B-offset,URL,vector,...,B_pos,pron_pos,A_idx,B_idx,pron_idx,A_vector,B_vector,pron_vector,A-coref,B-coref
0,development-1,Zoe Telford -- played the police officer girlf...,her,274,Cheryl Cassidy,191,Pauline,207,http://en.wikipedia.org/wiki/List_of_Teachers_...,"[[-0.30144718, -0.08635384, -0.20015159, -0.25...",...,0.608696,0.826087,"[54, 55, 56, 57, 58, 59]","[61, 62, 63]",[81],"[[0.14136104, -0.38602263, -0.19260919, 0.4973...","[[0.6761073, -0.45650303, -0.27585956, -0.3139...","[[-0.1408648, -0.009012785, -0.58574224, -0.96...",True,False
1,development-2,"He grew up in Evanston, Illinois the second ol...",His,284,MacKenzie,228,Bernard Leach,251,http://en.wikipedia.org/wiki/Warren_MacKenzie,"[[0.34895068, 0.6811254, 0.92398053, -0.417637...",...,0.333333,0.380952,"[63, 64, 65]","[68, 69, 70, 71, 72]",[78],"[[-0.75297713, -0.91880596, -0.31332776, 0.235...","[[-0.014572581, 0.3896196, -0.4235178, -0.2201...","[[-0.5127397, 0.38019675, -0.19245711, -0.6960...",True,False
2,development-3,"He had been reelected to Congress, but resigne...",his,265,Angeloz,173,De la Sota,246,http://en.wikipedia.org/wiki/Jos%C3%A9_Manuel_...,"[[-0.3016174, 0.06391026, -0.15569207, -0.1670...",...,0.505155,0.556701,"[43, 44]","[59, 60, 61, 62]",[65],"[[-0.44509542, -0.38031942, 0.01635306, 0.1235...","[[0.06467356, 0.14759275, 0.38993636, -0.09622...","[[-0.40798748, 0.8667205, -0.14823496, 0.31704...",False,True
3,development-4,The current members of Crime have also perform...,his,321,Hell,174,Henry Rosenthal,336,http://en.wikipedia.org/wiki/Crime_(band),"[[0.31311065, -0.4944703, 0.44825977, -0.12736...",...,0.657143,0.619048,[40],"[75, 76, 77, 78, 79, 80]",[71],"[[0.043780833, 0.44859782, 0.566811, -0.007530...","[[-0.40079117, 0.6081313, 0.008237168, -0.2687...","[[-0.23520903, -0.029176882, -0.047637727, 0.0...",False,True
4,development-5,Her Santa Fe Opera debut in 2005 was as Nuria ...,She,437,Kitty Oppenheimer,219,Rivera,294,http://en.wikipedia.org/wiki/Jessica_Rivera,"[[0.1210996, 0.19799706, 0.4914001, -0.4640768...",...,0.619565,0.923913,"[59, 60, 61, 62, 63]","[78, 79]",[116],"[[0.31352907, 0.8938127, -0.27805957, 0.610802...","[[0.60271496, 0.43095493, -0.86851233, 0.58121...","[[-0.7814366, -0.24526855, 0.24610633, -0.3730...",False,True


In [5]:
train_vector_ = np.concatenate(gap_train.vector,axis = 0)

In [6]:
mean_vector_1 = train_vector_.mean(axis = 0)
train_vector = train_vector_ - mean_vector_1

In [7]:
train_vector.shape

(223174, 1024)

In [8]:
import torch
import torch.nn as nn


batch_size = 256
learning_rate = 1e-3
steps = 5000

In [9]:
class autoencoder(nn.Module):
    def __init__(self,dim,hidden):
        super(autoencoder, self).__init__()
        self.encoder = nn.Linear(dim, hidden,bias = False)
        self.decoder = nn.Linear(hidden, dim,bias = False)

    def forward(self, x):
        h = self.encoder(x)
        x_ = self.decoder(h)
        return x_,h,x-x_

In [10]:
PPA = autoencoder(1024,4)
PPA.cuda()
criterion = nn.MSELoss()
criterion.cuda()
optimizer = torch.optim.Adam(
    PPA.parameters(), lr=learning_rate, weight_decay=1e-5)

for k in range(steps):
    pos = np.random.randint(0,train_vector.shape[0] - batch_size+1)
    data_train = torch.Tensor(train_vector[pos:pos+batch_size,:]).cuda()
    # ===================forward=====================
    data_reconstruction, _,_ = PPA(data_train)
    loss = criterion(data_train, data_reconstruction)
    # ===================backward====================
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # ===================log========================
    if steps%100 == 0:
        print('steps [{}/{}], loss:{:.4f}'.format(k + 1, steps, loss.item()))


torch.save(PPA.state_dict(), './temp_result/PPA.pth')

steps [1/5000], loss:0.3791
steps [2/5000], loss:0.3727
steps [3/5000], loss:0.3486
steps [4/5000], loss:0.3823
steps [5/5000], loss:0.3777
steps [6/5000], loss:0.3607
steps [7/5000], loss:0.4024
steps [8/5000], loss:0.3699
steps [9/5000], loss:0.3611
steps [10/5000], loss:0.3602
steps [11/5000], loss:0.3706
steps [12/5000], loss:0.3783
steps [13/5000], loss:0.3571
steps [14/5000], loss:0.3569
steps [15/5000], loss:0.3342
steps [16/5000], loss:0.3517
steps [17/5000], loss:0.3540
steps [18/5000], loss:0.3940
steps [19/5000], loss:0.3940
steps [20/5000], loss:0.3427
steps [21/5000], loss:0.3446
steps [22/5000], loss:0.3311
steps [23/5000], loss:0.3718
steps [24/5000], loss:0.3625
steps [25/5000], loss:0.3518
steps [26/5000], loss:0.3603
steps [27/5000], loss:0.3624
steps [28/5000], loss:0.3727
steps [29/5000], loss:0.3417
steps [30/5000], loss:0.3524
steps [31/5000], loss:0.3457
steps [32/5000], loss:0.3208
steps [33/5000], loss:0.3770
steps [34/5000], loss:0.3318
steps [35/5000], loss:0

steps [347/5000], loss:0.3242
steps [348/5000], loss:0.3187
steps [349/5000], loss:0.3384
steps [350/5000], loss:0.3519
steps [351/5000], loss:0.3577
steps [352/5000], loss:0.3049
steps [353/5000], loss:0.3002
steps [354/5000], loss:0.3503
steps [355/5000], loss:0.3222
steps [356/5000], loss:0.3437
steps [357/5000], loss:0.3265
steps [358/5000], loss:0.3612
steps [359/5000], loss:0.3372
steps [360/5000], loss:0.2996
steps [361/5000], loss:0.3369
steps [362/5000], loss:0.3103
steps [363/5000], loss:0.3409
steps [364/5000], loss:0.3745
steps [365/5000], loss:0.3395
steps [366/5000], loss:0.3262
steps [367/5000], loss:0.3293
steps [368/5000], loss:0.3788
steps [369/5000], loss:0.3445
steps [370/5000], loss:0.3346
steps [371/5000], loss:0.3416
steps [372/5000], loss:0.3304
steps [373/5000], loss:0.2920
steps [374/5000], loss:0.3425
steps [375/5000], loss:0.3646
steps [376/5000], loss:0.3170
steps [377/5000], loss:0.3206
steps [378/5000], loss:0.3279
steps [379/5000], loss:0.3226
steps [380

steps [732/5000], loss:0.2715
steps [733/5000], loss:0.2707
steps [734/5000], loss:0.2876
steps [735/5000], loss:0.2828
steps [736/5000], loss:0.3037
steps [737/5000], loss:0.2805
steps [738/5000], loss:0.2774
steps [739/5000], loss:0.3281
steps [740/5000], loss:0.2912
steps [741/5000], loss:0.2790
steps [742/5000], loss:0.2869
steps [743/5000], loss:0.2920
steps [744/5000], loss:0.3105
steps [745/5000], loss:0.2644
steps [746/5000], loss:0.3256
steps [747/5000], loss:0.3371
steps [748/5000], loss:0.2816
steps [749/5000], loss:0.2878
steps [750/5000], loss:0.2716
steps [751/5000], loss:0.2898
steps [752/5000], loss:0.3388
steps [753/5000], loss:0.3013
steps [754/5000], loss:0.2748
steps [755/5000], loss:0.2936
steps [756/5000], loss:0.2765
steps [757/5000], loss:0.2783
steps [758/5000], loss:0.2813
steps [759/5000], loss:0.2624
steps [760/5000], loss:0.2982
steps [761/5000], loss:0.3143
steps [762/5000], loss:0.3190
steps [763/5000], loss:0.3506
steps [764/5000], loss:0.2621
steps [765

steps [1128/5000], loss:0.2941
steps [1129/5000], loss:0.3053
steps [1130/5000], loss:0.2823
steps [1131/5000], loss:0.2811
steps [1132/5000], loss:0.3242
steps [1133/5000], loss:0.3022
steps [1134/5000], loss:0.2702
steps [1135/5000], loss:0.3002
steps [1136/5000], loss:0.2689
steps [1137/5000], loss:0.2725
steps [1138/5000], loss:0.2945
steps [1139/5000], loss:0.2865
steps [1140/5000], loss:0.3096
steps [1141/5000], loss:0.3480
steps [1142/5000], loss:0.2731
steps [1143/5000], loss:0.2774
steps [1144/5000], loss:0.2748
steps [1145/5000], loss:0.2807
steps [1146/5000], loss:0.2912
steps [1147/5000], loss:0.2787
steps [1148/5000], loss:0.2934
steps [1149/5000], loss:0.2924
steps [1150/5000], loss:0.2960
steps [1151/5000], loss:0.2755
steps [1152/5000], loss:0.3086
steps [1153/5000], loss:0.2708
steps [1154/5000], loss:0.3121
steps [1155/5000], loss:0.2894
steps [1156/5000], loss:0.2654
steps [1157/5000], loss:0.2781
steps [1158/5000], loss:0.2818
steps [1159/5000], loss:0.2928
steps [1

steps [1506/5000], loss:0.2751
steps [1507/5000], loss:0.2809
steps [1508/5000], loss:0.2592
steps [1509/5000], loss:0.2837
steps [1510/5000], loss:0.2938
steps [1511/5000], loss:0.2765
steps [1512/5000], loss:0.3104
steps [1513/5000], loss:0.2878
steps [1514/5000], loss:0.2900
steps [1515/5000], loss:0.2750
steps [1516/5000], loss:0.3133
steps [1517/5000], loss:0.2864
steps [1518/5000], loss:0.3234
steps [1519/5000], loss:0.2815
steps [1520/5000], loss:0.2710
steps [1521/5000], loss:0.2744
steps [1522/5000], loss:0.2904
steps [1523/5000], loss:0.3352
steps [1524/5000], loss:0.2973
steps [1525/5000], loss:0.2581
steps [1526/5000], loss:0.2585
steps [1527/5000], loss:0.2907
steps [1528/5000], loss:0.2914
steps [1529/5000], loss:0.2921
steps [1530/5000], loss:0.3039
steps [1531/5000], loss:0.2878
steps [1532/5000], loss:0.2689
steps [1533/5000], loss:0.3141
steps [1534/5000], loss:0.2885
steps [1535/5000], loss:0.3152
steps [1536/5000], loss:0.2633
steps [1537/5000], loss:0.2674
steps [1

...

steps [3396/5000], loss:0.2723
steps [3397/5000], loss:0.2898
steps [3398/5000], loss:0.2780
steps [3399/5000], loss:0.2763
steps [3400/5000], loss:0.2780
steps [3401/5000], loss:0.2701
steps [3402/5000], loss:0.3074
steps [3403/5000], loss:0.2895
steps [3404/5000], loss:0.2934
steps [3405/5000], loss:0.2971
steps [3406/5000], loss:0.2599
steps [3407/5000], loss:0.3086
steps [3408/5000], loss:0.2917
steps [3409/5000], loss:0.2798
steps [3410/5000], loss:0.3091
steps [3411/5000], loss:0.3114
steps [3412/5000], loss:0.2998
steps [3413/5000], loss:0.2530
steps [3414/5000], loss:0.2905
steps [3415/5000], loss:0.2573
steps [3416/5000], loss:0.2846
steps [3417/5000], loss:0.2778
steps [3418/5000], loss:0.2776
steps [3419/5000], loss:0.2867
steps [3420/5000], loss:0.3373
steps [3421/5000], loss:0.2977
steps [3422/5000], loss:0.2674
steps [3423/5000], loss:0.2532
steps [3424/5000], loss:0.3016
steps [3425/5000], loss:0.3111
steps [3426/5000], loss:0.2667
steps [3427/5000], loss:0.3379
steps [3

steps [3781/5000], loss:0.2669
steps [3782/5000], loss:0.3099
steps [3783/5000], loss:0.2916
steps [3784/5000], loss:0.2631
steps [3785/5000], loss:0.2646
steps [3786/5000], loss:0.2760
steps [3787/5000], loss:0.2819
steps [3788/5000], loss:0.2948
steps [3789/5000], loss:0.2700
steps [3790/5000], loss:0.2887
steps [3791/5000], loss:0.2866
steps [3792/5000], loss:0.2684
steps [3793/5000], loss:0.2679
steps [3794/5000], loss:0.2834
steps [3795/5000], loss:0.3094
steps [3796/5000], loss:0.3057
steps [3797/5000], loss:0.3404
steps [3798/5000], loss:0.2827
steps [3799/5000], loss:0.3260
steps [3800/5000], loss:0.2726
steps [3801/5000], loss:0.2754
steps [3802/5000], loss:0.3397
steps [3803/5000], loss:0.3070
steps [3804/5000], loss:0.2735
steps [3805/5000], loss:0.2840
steps [3806/5000], loss:0.3211
steps [3807/5000], loss:0.3039
steps [3808/5000], loss:0.2727
steps [3809/5000], loss:0.2933
steps [3810/5000], loss:0.2808
steps [3811/5000], loss:0.2938
steps [3812/5000], loss:0.2602
steps [3

steps [4166/5000], loss:0.2773
steps [4167/5000], loss:0.3102
steps [4168/5000], loss:0.2931
steps [4169/5000], loss:0.2594
steps [4170/5000], loss:0.2657
steps [4171/5000], loss:0.2602
steps [4172/5000], loss:0.2688
steps [4173/5000], loss:0.2847
steps [4174/5000], loss:0.2615
steps [4175/5000], loss:0.3132
steps [4176/5000], loss:0.2713
steps [4177/5000], loss:0.2745
steps [4178/5000], loss:0.2903
steps [4179/5000], loss:0.2819
steps [4180/5000], loss:0.3107
steps [4181/5000], loss:0.2820
steps [4182/5000], loss:0.2796
steps [4183/5000], loss:0.2787
steps [4184/5000], loss:0.2645
steps [4185/5000], loss:0.2671
steps [4186/5000], loss:0.2573
steps [4187/5000], loss:0.2845
steps [4188/5000], loss:0.2700
steps [4189/5000], loss:0.3338
steps [4190/5000], loss:0.2833
steps [4191/5000], loss:0.2710
steps [4192/5000], loss:0.2802
steps [4193/5000], loss:0.2730
steps [4194/5000], loss:0.3047
steps [4195/5000], loss:0.2672
steps [4196/5000], loss:0.2823
steps [4197/5000], loss:0.2787
steps [4

steps [4554/5000], loss:0.2992
steps [4555/5000], loss:0.2782
steps [4556/5000], loss:0.2881
steps [4557/5000], loss:0.3171
steps [4558/5000], loss:0.2831
steps [4559/5000], loss:0.2840
steps [4560/5000], loss:0.2890
steps [4561/5000], loss:0.2819
steps [4562/5000], loss:0.3175
steps [4563/5000], loss:0.2662
steps [4564/5000], loss:0.2760
steps [4565/5000], loss:0.2729
steps [4566/5000], loss:0.2692
steps [4567/5000], loss:0.3095
steps [4568/5000], loss:0.3013
steps [4569/5000], loss:0.3406
steps [4570/5000], loss:0.2934
steps [4571/5000], loss:0.3025
steps [4572/5000], loss:0.2684
steps [4573/5000], loss:0.2569
steps [4574/5000], loss:0.3128
steps [4575/5000], loss:0.3063
steps [4576/5000], loss:0.2481
steps [4577/5000], loss:0.2826
steps [4578/5000], loss:0.2813
steps [4579/5000], loss:0.3069
steps [4580/5000], loss:0.2863
steps [4581/5000], loss:0.3038
steps [4582/5000], loss:0.3042
steps [4583/5000], loss:0.2885
steps [4584/5000], loss:0.2909
steps [4585/5000], loss:0.3097
steps [4

steps [4927/5000], loss:0.3097
steps [4928/5000], loss:0.2869
steps [4929/5000], loss:0.2940
steps [4930/5000], loss:0.2828
steps [4931/5000], loss:0.3151
steps [4932/5000], loss:0.2748
steps [4933/5000], loss:0.2921
steps [4934/5000], loss:0.2932
steps [4935/5000], loss:0.2938
steps [4936/5000], loss:0.2635
steps [4937/5000], loss:0.2625
steps [4938/5000], loss:0.3053
steps [4939/5000], loss:0.3320
steps [4940/5000], loss:0.2656
steps [4941/5000], loss:0.2621
steps [4942/5000], loss:0.2779
steps [4943/5000], loss:0.2632
steps [4944/5000], loss:0.2730
steps [4945/5000], loss:0.2787
steps [4946/5000], loss:0.3239
steps [4947/5000], loss:0.3125
steps [4948/5000], loss:0.2783
steps [4949/5000], loss:0.2714
steps [4950/5000], loss:0.2971
steps [4951/5000], loss:0.2747
steps [4952/5000], loss:0.2673
steps [4953/5000], loss:0.3024
steps [4954/5000], loss:0.2885
steps [4955/5000], loss:0.2707
steps [4956/5000], loss:0.3080
steps [4957/5000], loss:0.2908
steps [4958/5000], loss:0.2981
steps [4

In [11]:
for df in [gap_train,gap_valid,gap_test]:
    for i in range(df.shape[0]):
        _,_,first_ppa_vector = PPA(torch.Tensor(df.vector[i]-mean_vector_1).cuda())
        first_ppa_vector = first_ppa_vector.cpu().data.numpy()
        df.loc[i,"vector"]  = [first_ppa_vector]
        df.loc[i,"A_vector"] = [first_ppa_vector[df.A_idx[i],:]]
        df.loc[i,"B_vector"] = [first_ppa_vector[df.B_idx[i],:]]
        df.loc[i,"pron_vector"] = [first_ppa_vector[df.pron_idx[i],:]]

In [12]:
train_vector.shape

(223174, 1024)

In [13]:
gap_train.to_pickle('./temp_result/train_kaggle_processed_PPA')
gap_test.to_pickle('./temp_result/test_kaggle_processed_PPA')
gap_valid.to_pickle('./temp_result/valid_kaggle_processed_PPA')

In [14]:
train_vector_ = np.concatenate(gap_train.vector,axis = 0)

In [15]:
mean_vector_2 = train_vector_.mean(axis = 0)
train_vector = train_vector_ - mean_vector_2

In [16]:
train_vector.shape

(223174, 1024)

In [17]:
PCA = autoencoder(1024,256)
PCA.cuda()
criterion = nn.MSELoss()
criterion.cuda()
optimizer = torch.optim.Adam(
    PCA.parameters(), lr=learning_rate, weight_decay=1e-5)

for k in range(steps):
    pos = np.random.randint(0,train_vector.shape[0] - batch_size+1)
    data_train = torch.Tensor(train_vector[pos:pos+batch_size,:]).cuda()
    # ===================forward=====================
    data_reconstruction, _,_ = PCA(data_train)
    loss = criterion(data_train, data_reconstruction)
    # ===================backward====================
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # ===================log========================
    if steps%100 == 0:
        print('steps [{}/{}], loss:{:.4f}'.format(k + 1, steps, loss.item()))


torch.save(PPA.state_dict(), './temp_result/PCA.pth')

steps [1/5000], loss:0.2843
steps [2/5000], loss:0.3183
steps [3/5000], loss:0.3177
steps [4/5000], loss:0.3191
steps [5/5000], loss:0.3196
steps [6/5000], loss:0.3078
steps [7/5000], loss:0.2641
steps [8/5000], loss:0.2538
steps [9/5000], loss:0.2772
steps [10/5000], loss:0.2854
steps [11/5000], loss:0.2699
steps [12/5000], loss:0.2694
steps [13/5000], loss:0.3374
steps [14/5000], loss:0.2557
steps [15/5000], loss:0.2596
steps [16/5000], loss:0.2635
steps [17/5000], loss:0.2620
steps [18/5000], loss:0.2859
steps [19/5000], loss:0.2350
steps [20/5000], loss:0.2421
steps [21/5000], loss:0.2585
steps [22/5000], loss:0.2345
steps [23/5000], loss:0.2430
steps [24/5000], loss:0.2348
steps [25/5000], loss:0.2456
steps [26/5000], loss:0.2414
steps [27/5000], loss:0.2256
steps [28/5000], loss:0.2273
steps [29/5000], loss:0.1954
steps [30/5000], loss:0.2553
steps [31/5000], loss:0.2173
steps [32/5000], loss:0.2282
steps [33/5000], loss:0.2122
steps [34/5000], loss:0.1806
steps [35/5000], loss:0

steps [367/5000], loss:0.1065
steps [368/5000], loss:0.0978
steps [369/5000], loss:0.1028
steps [370/5000], loss:0.0992
steps [371/5000], loss:0.1070
steps [372/5000], loss:0.1093
steps [373/5000], loss:0.1196
steps [374/5000], loss:0.1004
steps [375/5000], loss:0.0951
steps [376/5000], loss:0.1002
steps [377/5000], loss:0.1134
steps [378/5000], loss:0.0997
steps [379/5000], loss:0.1186
steps [380/5000], loss:0.0887
steps [381/5000], loss:0.0942
steps [382/5000], loss:0.0991
steps [383/5000], loss:0.1149
steps [384/5000], loss:0.0990
steps [385/5000], loss:0.1045
steps [386/5000], loss:0.1085
steps [387/5000], loss:0.1319
steps [388/5000], loss:0.0948
steps [389/5000], loss:0.1052
steps [390/5000], loss:0.0877
steps [391/5000], loss:0.1094
steps [392/5000], loss:0.0908
steps [393/5000], loss:0.1098
steps [394/5000], loss:0.1009
steps [395/5000], loss:0.0984
steps [396/5000], loss:0.1042
steps [397/5000], loss:0.1073
steps [398/5000], loss:0.0935
steps [399/5000], loss:0.0947
steps [400

steps [731/5000], loss:0.0925
steps [732/5000], loss:0.0892
steps [733/5000], loss:0.1045
steps [734/5000], loss:0.0992
steps [735/5000], loss:0.0851
steps [736/5000], loss:0.0953
steps [737/5000], loss:0.0913
steps [738/5000], loss:0.0878
steps [739/5000], loss:0.0882
steps [740/5000], loss:0.1022
steps [741/5000], loss:0.0958
steps [742/5000], loss:0.1036
steps [743/5000], loss:0.1027
steps [744/5000], loss:0.0999
steps [745/5000], loss:0.1002
steps [746/5000], loss:0.0903
steps [747/5000], loss:0.1165
steps [748/5000], loss:0.1053
steps [749/5000], loss:0.0918
steps [750/5000], loss:0.1238
steps [751/5000], loss:0.0980
steps [752/5000], loss:0.0931
steps [753/5000], loss:0.1148
steps [754/5000], loss:0.1049
steps [755/5000], loss:0.0945
steps [756/5000], loss:0.0880
steps [757/5000], loss:0.1097
steps [758/5000], loss:0.1104
steps [759/5000], loss:0.0958
steps [760/5000], loss:0.0960
steps [761/5000], loss:0.0853
steps [762/5000], loss:0.1001
steps [763/5000], loss:0.0946
steps [764

steps [1075/5000], loss:0.1066
steps [1076/5000], loss:0.0872
steps [1077/5000], loss:0.0954
steps [1078/5000], loss:0.0885
steps [1079/5000], loss:0.0965
steps [1080/5000], loss:0.0970
steps [1081/5000], loss:0.1021
steps [1082/5000], loss:0.1037
steps [1083/5000], loss:0.0955
steps [1084/5000], loss:0.1027
steps [1085/5000], loss:0.0992
steps [1086/5000], loss:0.0871
steps [1087/5000], loss:0.0869
steps [1088/5000], loss:0.0889
steps [1089/5000], loss:0.0962
steps [1090/5000], loss:0.0947
steps [1091/5000], loss:0.1309
steps [1092/5000], loss:0.1032
steps [1093/5000], loss:0.1039
steps [1094/5000], loss:0.0939
steps [1095/5000], loss:0.0979
steps [1096/5000], loss:0.0952
steps [1097/5000], loss:0.1015
steps [1098/5000], loss:0.0898
steps [1099/5000], loss:0.0985
steps [1100/5000], loss:0.0977
steps [1101/5000], loss:0.0941
steps [1102/5000], loss:0.1276
steps [1103/5000], loss:0.0901
steps [1104/5000], loss:0.1137
steps [1105/5000], loss:0.0934
steps [1106/5000], loss:0.0927
steps [1

steps [1451/5000], loss:0.0901
steps [1452/5000], loss:0.1182
steps [1453/5000], loss:0.1087
steps [1454/5000], loss:0.0939
steps [1455/5000], loss:0.0963
steps [1456/5000], loss:0.1044
steps [1457/5000], loss:0.1043
steps [1458/5000], loss:0.0886
steps [1459/5000], loss:0.1095
steps [1460/5000], loss:0.1116
steps [1461/5000], loss:0.0867
steps [1462/5000], loss:0.0978
steps [1463/5000], loss:0.0925
steps [1464/5000], loss:0.0815
steps [1465/5000], loss:0.0876
steps [1466/5000], loss:0.1072
steps [1467/5000], loss:0.1031
steps [1468/5000], loss:0.0932
steps [1469/5000], loss:0.0814
steps [1470/5000], loss:0.0916
steps [1471/5000], loss:0.1147
steps [1472/5000], loss:0.0993
steps [1473/5000], loss:0.0982
steps [1474/5000], loss:0.1020
steps [1475/5000], loss:0.0883
steps [1476/5000], loss:0.1182
steps [1477/5000], loss:0.0985
steps [1478/5000], loss:0.0968
steps [1479/5000], loss:0.0980
steps [1480/5000], loss:0.1006
steps [1481/5000], loss:0.0869
steps [1482/5000], loss:0.1034
steps [1

...

steps [3295/5000], loss:0.1027
steps [3296/5000], loss:0.1043
steps [3297/5000], loss:0.0818
steps [3298/5000], loss:0.0896
steps [3299/5000], loss:0.1010
steps [3300/5000], loss:0.0960
steps [3301/5000], loss:0.0837
steps [3302/5000], loss:0.0914
steps [3303/5000], loss:0.1053
steps [3304/5000], loss:0.0898
steps [3305/5000], loss:0.1022
steps [3306/5000], loss:0.0931
steps [3307/5000], loss:0.0942
steps [3308/5000], loss:0.1053
steps [3309/5000], loss:0.1029
steps [3310/5000], loss:0.0948
steps [3311/5000], loss:0.1028
steps [3312/5000], loss:0.0865
steps [3313/5000], loss:0.0880
steps [3314/5000], loss:0.1056
steps [3315/5000], loss:0.1108
steps [3316/5000], loss:0.0895
steps [3317/5000], loss:0.0965
steps [3318/5000], loss:0.1140
steps [3319/5000], loss:0.0863
steps [3320/5000], loss:0.0930
steps [3321/5000], loss:0.1098
steps [3322/5000], loss:0.0905
steps [3323/5000], loss:0.0962
steps [3324/5000], loss:0.0837
steps [3325/5000], loss:0.0895
steps [3326/5000], loss:0.0869
steps [3

steps [3676/5000], loss:0.1018
steps [3677/5000], loss:0.1052
steps [3678/5000], loss:0.1068
steps [3679/5000], loss:0.0893
steps [3680/5000], loss:0.0863
steps [3681/5000], loss:0.1043
steps [3682/5000], loss:0.0978
steps [3683/5000], loss:0.0881
steps [3684/5000], loss:0.0926
steps [3685/5000], loss:0.1018
steps [3686/5000], loss:0.1081
steps [3687/5000], loss:0.0929
steps [3688/5000], loss:0.0883
steps [3689/5000], loss:0.1048
steps [3690/5000], loss:0.1046
steps [3691/5000], loss:0.1087
steps [3692/5000], loss:0.0909
steps [3693/5000], loss:0.0932
steps [3694/5000], loss:0.1090
steps [3695/5000], loss:0.0871
steps [3696/5000], loss:0.1105
steps [3697/5000], loss:0.0868
steps [3698/5000], loss:0.0975
steps [3699/5000], loss:0.0899
steps [3700/5000], loss:0.0943
steps [3701/5000], loss:0.0984
steps [3702/5000], loss:0.0863
steps [3703/5000], loss:0.0984
steps [3704/5000], loss:0.0884
steps [3705/5000], loss:0.0938
steps [3706/5000], loss:0.0985
steps [3707/5000], loss:0.0957
steps [3

steps [4053/5000], loss:0.2688
steps [4054/5000], loss:0.1234
steps [4055/5000], loss:0.1236
steps [4056/5000], loss:0.1051
steps [4057/5000], loss:0.1183
steps [4058/5000], loss:0.1169
steps [4059/5000], loss:0.1209
steps [4060/5000], loss:0.1523
steps [4061/5000], loss:0.1114
steps [4062/5000], loss:0.1127
steps [4063/5000], loss:0.1102
steps [4064/5000], loss:0.1165
steps [4065/5000], loss:0.1263
steps [4066/5000], loss:0.1014
steps [4067/5000], loss:0.1144
steps [4068/5000], loss:0.1181
steps [4069/5000], loss:0.1272
steps [4070/5000], loss:0.1052
steps [4071/5000], loss:0.1289
steps [4072/5000], loss:0.1043
steps [4073/5000], loss:0.0936
steps [4074/5000], loss:0.0950
steps [4075/5000], loss:0.1013
steps [4076/5000], loss:0.1086
steps [4077/5000], loss:0.0924
steps [4078/5000], loss:0.0922
steps [4079/5000], loss:0.1040
steps [4080/5000], loss:0.1068
steps [4081/5000], loss:0.0958
steps [4082/5000], loss:0.1009
steps [4083/5000], loss:0.0961
steps [4084/5000], loss:0.1012
steps [4

steps [4419/5000], loss:0.0853
steps [4420/5000], loss:0.0858
steps [4421/5000], loss:0.0974
steps [4422/5000], loss:0.1023
steps [4423/5000], loss:0.0903
steps [4424/5000], loss:0.1010
steps [4425/5000], loss:0.0898
steps [4426/5000], loss:0.0927
steps [4427/5000], loss:0.0930
steps [4428/5000], loss:0.0842
steps [4429/5000], loss:0.0966
steps [4430/5000], loss:0.0831
steps [4431/5000], loss:0.0790
steps [4432/5000], loss:0.0910
steps [4433/5000], loss:0.1000
steps [4434/5000], loss:0.0879
steps [4435/5000], loss:0.1114
steps [4436/5000], loss:0.0803
steps [4437/5000], loss:0.0966
steps [4438/5000], loss:0.0860
steps [4439/5000], loss:0.1077
steps [4440/5000], loss:0.0834
steps [4441/5000], loss:0.0920
steps [4442/5000], loss:0.1081
steps [4443/5000], loss:0.0845
steps [4444/5000], loss:0.0882
steps [4445/5000], loss:0.0921
steps [4446/5000], loss:0.0991
steps [4447/5000], loss:0.0940
steps [4448/5000], loss:0.0970
steps [4449/5000], loss:0.0954
steps [4450/5000], loss:0.0901
steps [4

steps [4801/5000], loss:0.0765
steps [4802/5000], loss:0.0882
steps [4803/5000], loss:0.0939
steps [4804/5000], loss:0.0882
steps [4805/5000], loss:0.0960
steps [4806/5000], loss:0.1088
steps [4807/5000], loss:0.0828
steps [4808/5000], loss:0.0986
steps [4809/5000], loss:0.0825
steps [4810/5000], loss:0.0986
steps [4811/5000], loss:0.0934
steps [4812/5000], loss:0.0848
steps [4813/5000], loss:0.0924
steps [4814/5000], loss:0.1045
steps [4815/5000], loss:0.0947
steps [4816/5000], loss:0.0836
steps [4817/5000], loss:0.0934
steps [4818/5000], loss:0.0981
steps [4819/5000], loss:0.1025
steps [4820/5000], loss:0.1136
steps [4821/5000], loss:0.0899
steps [4822/5000], loss:0.1276
steps [4823/5000], loss:0.0884
steps [4824/5000], loss:0.0845
steps [4825/5000], loss:0.1161
steps [4826/5000], loss:0.0834
steps [4827/5000], loss:0.0962
steps [4828/5000], loss:0.1019
steps [4829/5000], loss:0.0900
steps [4830/5000], loss:0.0963
steps [4831/5000], loss:0.0905
steps [4832/5000], loss:0.0945
steps [4

In [18]:
for df in [gap_train,gap_valid,gap_test]:
    for i in range(df.shape[0]):
        _,pca_vector,_ = PCA(torch.Tensor(df.vector[i]-mean_vector_2).cuda())
        pca_vector = pca_vector.cpu().data.numpy()
        df.loc[i,"vector"]  = [pca_vector]
        df.loc[i,"A_vector"] = [pca_vector[df.A_idx[i],:]]
        df.loc[i,"B_vector"] = [pca_vector[df.B_idx[i],:]]
        df.loc[i,"pron_vector"] = [pca_vector[df.pron_idx[i],:]]

In [19]:
gap_train.to_pickle('./temp_result/train_kaggle_processed_PPA_PCA')
gap_test.to_pickle('./temp_result/test_kaggle_processed_PPA_PCA')
gap_valid.to_pickle('./temp_result/valid_kaggle_processed_PPA_PCA')

In [20]:
train_vector_ = np.concatenate(gap_train.vector,axis = 0)

In [21]:
mean_vector_3 = train_vector_.mean(axis = 0)
train_vector = train_vector_ - mean_vector_3

In [22]:
train_vector.shape

(223174, 256)

In [23]:
PPA = autoencoder(256,4)
PPA.cuda()
criterion = nn.MSELoss()
criterion.cuda()
optimizer = torch.optim.Adam(
    PPA.parameters(), lr=learning_rate, weight_decay=1e-5)

for k in range(steps):
    pos = np.random.randint(0,train_vector.shape[0] - batch_size+1)
    data_train = torch.Tensor(train_vector[pos:pos+batch_size,:]).cuda()
    # ===================forward=====================
    data_reconstruction, _,_ = PPA(data_train)
    loss = criterion(data_train, data_reconstruction)
    # ===================backward====================
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # ===================log========================
    if steps%100 == 0:
        print('steps [{}/{}], loss:{:.4f}'.format(k + 1, steps, loss.item()))


torch.save(PPA.state_dict(), './temp_result/PPA2.pth')

steps [1/5000], loss:0.4944
steps [2/5000], loss:0.4163
steps [3/5000], loss:0.4119
steps [4/5000], loss:0.4183
steps [5/5000], loss:0.4871
steps [6/5000], loss:0.4139
steps [7/5000], loss:0.4203
steps [8/5000], loss:0.4639
steps [9/5000], loss:0.4404
steps [10/5000], loss:0.4345
steps [11/5000], loss:0.4141
steps [12/5000], loss:0.4447
steps [13/5000], loss:0.4379
steps [14/5000], loss:0.4203
steps [15/5000], loss:0.4740
steps [16/5000], loss:0.4577
steps [17/5000], loss:0.4538
steps [18/5000], loss:0.4556
steps [19/5000], loss:0.4635
steps [20/5000], loss:0.4402
steps [21/5000], loss:0.4166
steps [22/5000], loss:0.5054
steps [23/5000], loss:0.4420
steps [24/5000], loss:0.4089
steps [25/5000], loss:0.4587
steps [26/5000], loss:0.4940
steps [27/5000], loss:0.4164
steps [28/5000], loss:0.3875
steps [29/5000], loss:0.5008
steps [30/5000], loss:0.3740
steps [31/5000], loss:0.4247
steps [32/5000], loss:0.3975
steps [33/5000], loss:0.3846
steps [34/5000], loss:0.4411
steps [35/5000], loss:0

steps [458/5000], loss:0.4458
steps [459/5000], loss:0.3956
steps [460/5000], loss:0.4097
steps [461/5000], loss:0.4134
steps [462/5000], loss:0.3648
steps [463/5000], loss:0.3462
steps [464/5000], loss:0.3989
steps [465/5000], loss:0.4312
steps [466/5000], loss:0.3772
steps [467/5000], loss:0.3750
steps [468/5000], loss:0.3609
steps [469/5000], loss:0.4072
steps [470/5000], loss:0.4073
steps [471/5000], loss:0.4549
steps [472/5000], loss:0.4116
steps [473/5000], loss:0.3968
steps [474/5000], loss:0.4120
steps [475/5000], loss:0.3932
steps [476/5000], loss:0.4098
steps [477/5000], loss:0.3566
steps [478/5000], loss:0.3658
steps [479/5000], loss:0.3676
steps [480/5000], loss:0.3794
steps [481/5000], loss:0.4464
steps [482/5000], loss:0.4120
steps [483/5000], loss:0.3751
steps [484/5000], loss:0.3844
steps [485/5000], loss:0.4239
steps [486/5000], loss:0.4066
steps [487/5000], loss:0.4028
steps [488/5000], loss:0.3422
steps [489/5000], loss:0.4087
steps [490/5000], loss:0.4549
steps [491

steps [923/5000], loss:0.4366
steps [924/5000], loss:0.4446
steps [925/5000], loss:0.3648
steps [926/5000], loss:0.3735
steps [927/5000], loss:0.3494
steps [928/5000], loss:0.3948
steps [929/5000], loss:0.3461
steps [930/5000], loss:0.4097
steps [931/5000], loss:0.4030
steps [932/5000], loss:0.3840
steps [933/5000], loss:0.3629
steps [934/5000], loss:0.4439
steps [935/5000], loss:0.3564
steps [936/5000], loss:0.4162
steps [937/5000], loss:0.4008
steps [938/5000], loss:0.4343
steps [939/5000], loss:0.3554
steps [940/5000], loss:0.4112
steps [941/5000], loss:0.3816
steps [942/5000], loss:0.4160
steps [943/5000], loss:0.4701
steps [944/5000], loss:0.3573
steps [945/5000], loss:0.3852
steps [946/5000], loss:0.3817
steps [947/5000], loss:0.4201
steps [948/5000], loss:0.3754
steps [949/5000], loss:0.3618
steps [950/5000], loss:0.4171
steps [951/5000], loss:0.4196
steps [952/5000], loss:0.3376
steps [953/5000], loss:0.4389
steps [954/5000], loss:0.3735
steps [955/5000], loss:0.3888
steps [956

steps [1381/5000], loss:0.3821
steps [1382/5000], loss:0.4025
steps [1383/5000], loss:0.4395
steps [1384/5000], loss:0.3424
steps [1385/5000], loss:0.3947
steps [1386/5000], loss:0.3842
steps [1387/5000], loss:0.4232
steps [1388/5000], loss:0.4128
steps [1389/5000], loss:0.3806
steps [1390/5000], loss:0.3607
steps [1391/5000], loss:0.3643
steps [1392/5000], loss:0.4396
steps [1393/5000], loss:0.4120
steps [1394/5000], loss:0.3731
steps [1395/5000], loss:0.4003
steps [1396/5000], loss:0.3581
steps [1397/5000], loss:0.3431
steps [1398/5000], loss:0.4008
steps [1399/5000], loss:0.3843
steps [1400/5000], loss:0.3884
steps [1401/5000], loss:0.4268
steps [1402/5000], loss:0.3980
steps [1403/5000], loss:0.3911
steps [1404/5000], loss:0.3968
steps [1405/5000], loss:0.3670
steps [1406/5000], loss:0.3806
steps [1407/5000], loss:0.3568
steps [1408/5000], loss:0.4541
steps [1409/5000], loss:0.3735
steps [1410/5000], loss:0.4597
steps [1411/5000], loss:0.3919
steps [1412/5000], loss:0.3691
steps [1

steps [1859/5000], loss:0.4091
steps [1860/5000], loss:0.3964
steps [1861/5000], loss:0.3556
steps [1862/5000], loss:0.3615
steps [1863/5000], loss:0.3721
steps [1864/5000], loss:0.3840
steps [1865/5000], loss:0.3722
steps [1866/5000], loss:0.4047
steps [1867/5000], loss:0.3771
steps [1868/5000], loss:0.3648
steps [1869/5000], loss:0.3400
steps [1870/5000], loss:0.3962
steps [1871/5000], loss:0.3681
steps [1872/5000], loss:0.3734
steps [1873/5000], loss:0.3536
steps [1874/5000], loss:0.3458
steps [1875/5000], loss:0.3949
steps [1876/5000], loss:0.4808
steps [1877/5000], loss:0.3921
steps [1878/5000], loss:0.4460
steps [1879/5000], loss:0.3775
steps [1880/5000], loss:0.4329
steps [1881/5000], loss:0.4095
steps [1882/5000], loss:0.5079
steps [1883/5000], loss:0.3561
steps [1884/5000], loss:0.3997
steps [1885/5000], loss:0.4085
steps [1886/5000], loss:0.3823
steps [1887/5000], loss:0.3541
steps [1888/5000], loss:0.3892
steps [1889/5000], loss:0.3488
steps [1890/5000], loss:0.3869
steps [1

...

steps [2809/5000], loss:0.3766
steps [2810/5000], loss:0.3988
steps [2811/5000], loss:0.3541
steps [2812/5000], loss:0.3830
steps [2813/5000], loss:0.3504
steps [2814/5000], loss:0.4105
steps [2815/5000], loss:0.3710
steps [2816/5000], loss:0.3688
steps [2817/5000], loss:0.3995
steps [2818/5000], loss:0.4543
steps [2819/5000], loss:0.4456
steps [2820/5000], loss:0.3963
steps [2821/5000], loss:0.3715
steps [2822/5000], loss:0.4094
steps [2823/5000], loss:0.3995
steps [2824/5000], loss:0.4057
steps [2825/5000], loss:0.3619
steps [2826/5000], loss:0.3572
steps [2827/5000], loss:0.4287
steps [2828/5000], loss:0.3523
steps [2829/5000], loss:0.4036
steps [2830/5000], loss:0.3727
steps [2831/5000], loss:0.3782
steps [2832/5000], loss:0.3729
steps [2833/5000], loss:0.4383
steps [2834/5000], loss:0.3872
steps [2835/5000], loss:0.3970
steps [2836/5000], loss:0.3643
steps [2837/5000], loss:0.3740
steps [2838/5000], loss:0.3941
steps [2839/5000], loss:0.3792
steps [2840/5000], loss:0.3905
steps [2

steps [3289/5000], loss:0.3774
steps [3290/5000], loss:0.3879
steps [3291/5000], loss:0.4014
steps [3292/5000], loss:0.4187
steps [3293/5000], loss:0.3645
steps [3294/5000], loss:0.3467
steps [3295/5000], loss:0.4302
steps [3296/5000], loss:0.3830
steps [3297/5000], loss:0.3612
steps [3298/5000], loss:0.4176
steps [3299/5000], loss:0.4191
steps [3300/5000], loss:0.4457
steps [3301/5000], loss:0.3531
steps [3302/5000], loss:0.3716
steps [3303/5000], loss:0.3480
steps [3304/5000], loss:0.3766
steps [3305/5000], loss:0.3763
steps [3306/5000], loss:0.3778
steps [3307/5000], loss:0.3566
steps [3308/5000], loss:0.4713
steps [3309/5000], loss:0.3386
steps [3310/5000], loss:0.3515
steps [3311/5000], loss:0.4107
steps [3312/5000], loss:0.3792
steps [3313/5000], loss:0.4395
steps [3314/5000], loss:0.4067
steps [3315/5000], loss:0.3721
steps [3316/5000], loss:0.3862
steps [3317/5000], loss:0.3708
steps [3318/5000], loss:0.3508
steps [3319/5000], loss:0.3552
steps [3320/5000], loss:0.3655
steps [3

steps [3780/5000], loss:0.3514
steps [3781/5000], loss:0.4303
steps [3782/5000], loss:0.3706
steps [3783/5000], loss:0.3430
steps [3784/5000], loss:0.4183
steps [3785/5000], loss:0.3562
steps [3786/5000], loss:0.3723
steps [3787/5000], loss:0.3837
steps [3788/5000], loss:0.4063
steps [3789/5000], loss:0.4350
steps [3790/5000], loss:0.3674
steps [3791/5000], loss:0.3817
steps [3792/5000], loss:0.3573
steps [3793/5000], loss:0.3637
steps [3794/5000], loss:0.3721
steps [3795/5000], loss:0.3375
steps [3796/5000], loss:0.3445
steps [3797/5000], loss:0.4084
steps [3798/5000], loss:0.3543
steps [3799/5000], loss:0.3543
steps [3800/5000], loss:0.3575
steps [3801/5000], loss:0.3808
steps [3802/5000], loss:0.3982
steps [3803/5000], loss:0.4411
steps [3804/5000], loss:0.3807
steps [3805/5000], loss:0.3954
steps [3806/5000], loss:0.3902
steps [3807/5000], loss:0.3540
steps [3808/5000], loss:0.3547
steps [3809/5000], loss:0.3694
steps [3810/5000], loss:0.4008
steps [3811/5000], loss:0.4137
steps [3

steps [4258/5000], loss:0.4220
steps [4259/5000], loss:0.3928
steps [4260/5000], loss:0.4061
steps [4261/5000], loss:0.5439
steps [4262/5000], loss:0.4589
steps [4263/5000], loss:0.3742
steps [4264/5000], loss:0.3953
steps [4265/5000], loss:0.3931
steps [4266/5000], loss:0.4065
steps [4267/5000], loss:0.4370
steps [4268/5000], loss:0.4043
steps [4269/5000], loss:0.3568
steps [4270/5000], loss:0.3794
steps [4271/5000], loss:0.3705
steps [4272/5000], loss:0.4297
steps [4273/5000], loss:0.4186
steps [4274/5000], loss:0.3461
steps [4275/5000], loss:0.3623
steps [4276/5000], loss:0.3796
steps [4277/5000], loss:0.3323
steps [4278/5000], loss:0.3603
steps [4279/5000], loss:0.3739
steps [4280/5000], loss:0.3960
steps [4281/5000], loss:0.3726
steps [4282/5000], loss:0.3720
steps [4283/5000], loss:0.3775
steps [4284/5000], loss:0.4626
steps [4285/5000], loss:0.4358
steps [4286/5000], loss:0.3989
steps [4287/5000], loss:0.3685
steps [4288/5000], loss:0.4032
steps [4289/5000], loss:0.4035
steps [4

steps [4730/5000], loss:0.3749
steps [4731/5000], loss:0.4058
steps [4732/5000], loss:0.3834
steps [4733/5000], loss:0.3873
steps [4734/5000], loss:0.3707
steps [4735/5000], loss:0.4027
steps [4736/5000], loss:0.3756
steps [4737/5000], loss:0.3549
steps [4738/5000], loss:0.3672
steps [4739/5000], loss:0.3953
steps [4740/5000], loss:0.3941
steps [4741/5000], loss:0.3465
steps [4742/5000], loss:0.3581
steps [4743/5000], loss:0.3609
steps [4744/5000], loss:0.3993
steps [4745/5000], loss:0.3519
steps [4746/5000], loss:0.3733
steps [4747/5000], loss:0.4235
steps [4748/5000], loss:0.3733
steps [4749/5000], loss:0.3919
steps [4750/5000], loss:0.3586
steps [4751/5000], loss:0.3516
steps [4752/5000], loss:0.3634
steps [4753/5000], loss:0.3712
steps [4754/5000], loss:0.3733
steps [4755/5000], loss:0.3817
steps [4756/5000], loss:0.3848
steps [4757/5000], loss:0.3662
steps [4758/5000], loss:0.3937
steps [4759/5000], loss:0.3911
steps [4760/5000], loss:0.3677
steps [4761/5000], loss:0.3422
steps [4

In [24]:
for df in [gap_train,gap_valid,gap_test]:
    for i in range(df.shape[0]):
        _,_,final_ppa_vector = PPA(torch.Tensor(df.vector[i]-mean_vector_3).cuda())
        final_ppa_vector = final_ppa_vector.cpu().data.numpy()
        df.loc[i,"vector"]  = [final_ppa_vector]
        df.loc[i,"A_vector"] = [final_ppa_vector[df.A_idx[i],:]]
        df.loc[i,"B_vector"] = [final_ppa_vector[df.B_idx[i],:]]
        df.loc[i,"pron_vector"] = [final_ppa_vector[df.pron_idx[i],:]]

In [25]:
gap_train.to_pickle('./temp_result/train_kaggle_processed_PPA_PCA_PPA')
gap_test.to_pickle('./temp_result/test_kaggle_processed_PPA_PCA_PPA')
gap_valid.to_pickle('./temp_result/valid_kaggle_processed_PPA_PCA_PPA')

In [26]:
import pickle
f = open( "./temp_result/mean_data", "wb" )
pickle.dump(mean_vector_1,  f)
pickle.dump(mean_vector_2,  f)
pickle.dump(mean_vector_3,  f)
f.close()